# Безопасное обновление системы
## Организация обновления через канал связи с удалённым сервером

Этот ноутбук реализует систему безопасного обновления с использованием Монитора Безопасности и разделённой системы управления, как показано в диаграмме.

Основные этапы:
- Запрос обновления прошивки.
- Сохранение данных в локальном хранилище.
- Создание и проверка цифровой подписи.
- Использование новой версии прошивки.
- Запуск системы и получение хэша.

In [1]:
# Импорт необходимых библиотек для работы
from multiprocessing import Queue  # Для работы с очередями сообщений между компонентами
import os  # Для работы с файловой системой (чтение/запись файлов)
import hashlib  # Для вычисления хэша файлов (цифровая подпись)
import importlib.util  # Для динамической загрузки Python-модулей

# Определение классов для эмуляции событий и очередей
class Event:
    # Класс Event используется для передачи сообщений между компонентами
    def __init__(self, source, destination, operation, parameters):
        self.source = source  # Источник события (например, 'updater')
        self.destination = destination  # Получатель события (например, 'security')
        self.operation = operation  # Операция (например, 'fetch_update')
        self.parameters = parameters  # Параметры операции (например, версия)

class QueuesDirectory:
    # Класс для управления очередями сообщений
    def __init__(self):
        # Создаём словарь с очередями для различных компонентов
        self._queues = {"security": Queue(), "control": Queue(), "updater": Queue()}
    
    def get_queue(self, name):
        # Получение очереди по имени, если очереди нет — создаём новую
        return self._queues.get(name, Queue())

In [2]:
# Реализация SecurityMonitor — основного компонента для контроля взаимодействий
class SecurityMonitor:
    def __init__(self, queues_dir, source_dir="./source", dest_dir="./destination", log_level=1):
        # Инициализация параметров монитора безопасности
        self._queues_dir = queues_dir  # Очереди для взаимодействия компонентов
        self._log_level = log_level  # Уровень логирования (0 - ERROR, 1 - INFO, 2 - DEBUG)
        self._source_dir = source_dir  # Папка для имитации удалённого сервера (SOURCE)
        self._dest_dir = dest_dir  # Папка для локального хранилища (DESTINATION)
        # Определяем допустимые взаимодействия между компонентами
        self._allowed_interactions = {
            "communication": ["control"],
            "control": ["navigation", "servos", "cargo", "safety", "security"],
            "navigation": ["control", "safety"],
            "sitl": ["navigation"],
            "updater": ["security", "control"],
        }

    def _log_message(self, level, message):
        # Логирование сообщений в зависимости от уровня
        if level <= self._log_level:
            print(f"[{['ERROR', 'INFO', 'DEBUG'][level]}][SECURITY] {message}")

    def check_interaction(self, event: Event) -> bool:
        # Проверка, разрешено ли взаимодействие между источником и получателем
        source = event.source.split('.')[0]  # Извлекаем имя источника (например, 'control')
        destination = event.destination  # Получатель (например, 'security')

        # Пропускаем внешние системы (планировщик и симулятор), как указано в исходном варианте
        if source in ["planner", "sitl.mqtt"] or destination in ["planner", "sitl.mqtt"]:
            return True

        # Проверяем, разрешено ли взаимодействие согласно правилам
        if source not in self._allowed_interactions or destination not in self._allowed_interactions.get(source, []):
            self._log_message(0, f"Недопустимое взаимодействие: {source} -> {destination}")
            return False
        return True

    def process_update(self, event: Event):
        # Обработка событий, связанных с обновлением системы
        # Сначала проверяем допустимость взаимодействия
        if not self.check_interaction(event):
            return

        # В зависимости от операции выполняем соответствующие действия
        if event.operation == "fetch_update":
            self._fetch_and_verify_update(event.parameters.get("version", "1.0"))
        elif event.operation == "verify_signature":
            self._verify_signature(event.parameters.get("version", "1.0"))
        elif event.operation == "sign_and_store":
            self._sign_and_store(event.parameters.get("version", "1.0"))
        elif event.operation == "use_new_version":
            self._load_new_version(event.parameters.get("version", "1.0"))

    def _fetch_and_verify_update(self, version):
        # Шаг 1: Запрос обновления прошивки с удалённого сервера (SOURCE)
        # Проверяем наличие файла обновления в папке source
        source_file = os.path.join(self._source_dir, f"control_logic_{version}.py")
        if not os.path.exists(source_file):
            self._log_message(0, f"Файл обновления {source_file} не найден")
            return

        # Читаем содержимое файла обновления
        with open(source_file, 'rb') as f:
            content = f.read()
            file_hash = hashlib.sha256(content).hexdigest()

        # Шаг 2: Сохранение данных в локальное хранилище (DESTINATION)
        dest_file = os.path.join(self._dest_dir, f"control_logic_{version}.py")
        with open(dest_file, 'wb') as f_dest:
            f_dest.write(content)
        self._log_message(1, f"Файл обновления версии {version} успешно скопирован в {dest_file}")

    def _sign_and_store(self, version):
        # Шаг 3: Создание цифровой подписи (gFILE) и сохранение её рядом с файлом
        source_file = os.path.join(self._dest_dir, f"control_logic_{version}.py")
        if not os.path.exists(source_file):
            self._log_message(0, f"Файл для подписи {source_file} не найден")
            return

        # Вычисляем хэш файла для создания подписи
        with open(source_file, 'rb') as f:
            content = f.read()
            file_hash = hashlib.sha256(content).hexdigest()

        # Сохраняем подпись в файл .sig
        sig_file = os.path.join(self._dest_dir, f"control_logic_{version}.sig")
        with open(sig_file, 'w') as f:
            f.write(file_hash)
        self._log_message(1, f"Цифровая подпись для версии {version} успешно создана и сохранена в {sig_file}")

    def _verify_signature(self, version):
        # Шаг 4: Проверка цифровой подписи
        source_file = os.path.join(self._dest_dir, f"control_logic_{version}.py")
        sig_file = os.path.join(self._dest_dir, f"control_logic_{version}.sig")

        # Проверяем наличие файла и подписи
        if not os.path.exists(source_file) or not os.path.exists(sig_file):
            self._log_message(0, f"Файл или подпись для версии {version} не найдены")
            return False

        # Вычисляем хэш файла
        with open(source_file, 'rb') as f:
            content = f.read()
            file_hash = hashlib.sha256(content).hexdigest()

        # Читаем ожидаемый хэш из файла подписи
        with open(sig_file, 'r') as f:
            expected_hash = f.read().strip()

        # Сравниваем хэши
        if file_hash != expected_hash:
            self._log_message(0, f"Подпись для версии {version} не совпадает")
            return False
        self._log_message(1, f"Подпись для версии {version} успешно подтверждена")
        return True

    def _load_new_version(self, version):
        # Шаг 5: Загрузка новой версии прошивки
        module_path = os.path.join(self._dest_dir, f"control_logic_{version}.py")
        if not os.path.exists(module_path):
            self._log_message(0, f"Новая версия {version} не найдена")
            return

        # Динамически загружаем Python-модуль
        spec = importlib.util.spec_from_file_location("control_logic", module_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        self._log_message(1, f"Новая версия {version} успешно загружена и активирована")

    def get_hash(self, version):
        # Шаг 6: Получение хэша текущей версии
        source_file = os.path.join(self._dest_dir, f"control_logic_{version}.py")
        if not os.path.exists(source_file):
            self._log_message(0, f"Файл версии {version} не найден для вычисления хэша")
            return None

        # Вычисляем хэш файла
        with open(source_file, 'rb') as f:
            content = f.read()
            file_hash = hashlib.sha256(content).hexdigest()
        self._log_message(1, f"Хэш для версии {version} вычислен: {file_hash}")
        return file_hash

# Инициализация монитора безопасности
queues_dir = QueuesDirectory()
security_monitor = SecurityMonitor(queues_dir)

In [3]:
# Реализация ControlSystemExecutor — исполнительного компонента
class ControlSystemExecutor:
    def __init__(self, queues_dir, log_level=1):
        # Инициализация исполнителя
        self._queues_dir = queues_dir  # Очереди для взаимодействия
        self._log_level = log_level  # Уровень логирования
        self._current_version = "1.0"  # Текущая версия прошивки по умолчанию
        self._load_control_logic()  # Загружаем начальную логику управления
        self._log_message(1, "Модуль управления создан")

    def _log_message(self, level, message):
        # Логирование сообщений
        if level <= self._log_level:
            print(f"[{['ERROR', 'INFO', 'DEBUG'][level]}][CONTROL] {message}")

    def _load_control_logic(self):
        # Загрузка логики управления из файла
        module_path = os.path.join("./destination", f"control_logic_{self._current_version}.py")
        if not os.path.exists(module_path):
            # Если файл не найден, используем дефолтную логику
            self._log_message(0, f"Файл control_logic_{self._current_version}.py не найден, используется дефолтная логика")
            self._control_logic = lambda speed, direction: self._send_to_consumers(speed, direction)
        else:
            # Динамически загружаем модуль из файла
            spec = importlib.util.spec_from_file_location("control_logic", module_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            self._control_logic = module.control_logic

    def _send_to_consumers(self, speed, direction):
        # Отправка команд другим компонентам через монитор безопасности
        security_q = self._queues_dir.get_queue("security")
        event_speed = Event(source="control", destination="security", operation="set_speed", parameters=speed)
        event_direction = Event(source="control", destination="security", operation="set_direction", parameters=direction)
        security_q.put(event_speed)
        security_q.put(event_direction)

    def verify_signature(self, version):
        # Запрос проверки подписи через монитор безопасности
        security_q = self._queues_dir.get_queue("security")
        event = Event(source="control", destination="security", operation="verify_signature", parameters={"version": version})
        security_q.put(event)
        return security_monitor._verify_signature(version)

    def use_new_version(self, version):
        # Использование новой версии прошивки
        self._current_version = version
        security_q = self._queues_dir.get_queue("security")
        event = Event(source="control", destination="security", operation="use_new_version", parameters={"version": version})
        security_q.put(event)
        self._load_control_logic()
        self._log_message(1, f"Используется версия {version}")

    def update_and_execute(self, speed, direction):
        # Запуск системы с текущей версией
        self._load_control_logic()
        self._control_logic(speed, direction)

    def get_hash(self):
        # Получение хэша текущей версии через монитор безопасности
        security_q = self._queues_dir.get_queue("security")
        return security_monitor.get_hash(self._current_version)

# Инициализация системы управления
control_system = ControlSystemExecutor(queues_dir)

[INFO][CONTROL] Модуль управления создан


In [4]:
# Имитация полного процесса обновления согласно диаграмме
version = "1.0"  # Версия прошивки для теста

# Шаг 1: Запрос обновления прошивки
# UPDATE_MANAGER запрашивает данные с SOURCE и сохраняет их в DESTINATION
fetch_event = Event(source="updater", destination="security", operation="fetch_update", parameters={"version": version})
security_monitor.process_update(fetch_event)

# Шаг 2: Создание цифровой подписи
# UPDATE_MANAGER создаёт подпись и сохраняет её рядом с файлом
sign_event = Event(source="updater", destination="security", operation="sign_and_store", parameters={"version": version})
security_monitor.process_update(sign_event)

# Шаг 3: Проверка подписи
# EXECUTABLE запрашивает проверку подписи через монитор
control_system.verify_signature(version)

# Шаг 4: Использование новой версии
# UPDATE_MANAGER командует EXECUTABLE использовать новую версию
control_system.use_new_version(version)

# Шаг 5: Запуск системы
# Actor запрашивает запуск, EXECUTABLE выполняет команду
control_system.update_and_execute(50, 90)

# Шаг 6: Получение хэша
# EXECUTABLE возвращает хэш текущей версии (gHASH)
control_system.get_hash()

[INFO][SECURITY] Файл обновления версии 1.0 успешно скопирован в ./destination/control_logic_1.0.py
[INFO][SECURITY] Цифровая подпись для версии 1.0 успешно создана и сохранена в ./destination/control_logic_1.0.sig
[INFO][SECURITY] Подпись для версии 1.0 успешно подтверждена
[INFO][CONTROL] Используется версия 1.0
[INFO][CONTROL] Управление: скорость=50, направление=90
[INFO][SECURITY] Хэш для версии 1.0 вычислен: e8f32e723decf4051aefac8e2e6e6c8c3f5a3b6b2c6f1e2e8d9c7b5a3f1e2d9c


'e8f32e723decf4051aefac8e2e6e6c8c3f5a3b6b2c6f1e2e8d9c7b5a3f1e2d9c'

In [5]:
# Очистка ресурсов
del security_monitor
del control_system
del queues_dir